In [ ]:
import re
import os
import json

In [ ]:
contractions = {
    "aint": "ain't", "arent": "aren't", "cant": "can't", "couldve":
    "could've", "couldnt": "couldn't", "couldn'tve": "couldn't've",
    "couldnt've": "couldn't've", "didnt": "didn't", "doesnt":
    "doesn't", "dont": "don't", "hadnt": "hadn't", "hadnt've":
    "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent":
    "haven't", "hed": "he'd", "hed've": "he'd've", "he'dve":
    "he'd've", "hes": "he's", "howd": "how'd", "howll": "how'll",
    "hows": "how's", "Id've": "I'd've", "I'dve": "I'd've", "Im":
    "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've":
    "it'd've", "it'dve": "it'd've", "itll": "it'll", "let's": "let's",
    "maam": "ma'am", "mightnt": "mightn't", "mightnt've":
    "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've",
    "mustnt": "mustn't", "mustve": "must've", "neednt": "needn't",
    "notve": "not've", "oclock": "o'clock", "oughtnt": "oughtn't",
    "ow's'at": "'ow's'at", "'ows'at": "'ow's'at", "'ow'sat":
    "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve":
    "she'd've", "she's": "she's", "shouldve": "should've", "shouldnt":
    "shouldn't", "shouldnt've": "shouldn't've", "shouldn'tve":
    "shouldn't've", "somebody'd": "somebodyd", "somebodyd've":
    "somebody'd've", "somebody'dve": "somebody'd've", "somebodyll":
    "somebody'll", "somebodys": "somebody's", "someoned": "someone'd",
    "someoned've": "someone'd've", "someone'dve": "someone'd've",
    "someonell": "someone'll", "someones": "someone's", "somethingd":
    "something'd", "somethingd've": "something'd've", "something'dve":
    "something'd've", "somethingll": "something'll", "thats":
    "that's", "thered": "there'd", "thered've": "there'd've",
    "there'dve": "there'd've", "therere": "there're", "theres":
    "there's", "theyd": "they'd", "theyd've": "they'd've", "they'dve":
    "they'd've", "theyll": "they'll", "theyre": "they're", "theyve":
    "they've", "twas": "'twas", "wasnt": "wasn't", "wed've":
    "we'd've", "we'dve": "we'd've", "weve": "we've", "werent":
    "weren't", "whatll": "what'll", "whatre": "what're", "whats":
    "what's", "whatve": "what've", "whens": "when's", "whered":
    "where'd", "wheres": "where's", "whereve": "where've", "whod":
    "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl":
    "who'll", "whos": "who's", "whove": "who've", "whyll": "why'll",
    "whyre": "why're", "whys": "why's", "wont": "won't", "wouldve":
    "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've",
    "wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll":
    "y'all'll", "y'allll": "y'all'll", "yall'd've": "y'all'd've",
    "y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", "youd":
    "you'd", "youd've": "you'd've", "you'dve": "you'd've", "youll":
    "you'll", "youre": "you're", "youve": "you've"
}

manual_map = {'none': '0',
              'zero': '0',
              'one': '1',
              'two': '2',
              'three': '3',
              'four': '4',
              'five': '5',
              'six': '6',
              'seven': '7',
              'eight': '8',
               'nine': '9',
              'ten': '10'}
articles = ['a', 'an', 'the']
period_strip = re.compile("(?!<=\d)(\.)(?!\d)")
comma_strip = re.compile("(\d)(\,)(\d)")
punct = [';', r"/", '[', ']', '"', '{', '}',
                '(', ')', '=', '+', '\\', '_', '-',
                '>', '<', '@', '`', ',', '?', '!']

def process_punctuation(inText):
    outText = inText
    for p in punct:
        if (p + ' ' in inText or ' ' + p in inText) \
           or (re.search(comma_strip, inText) != None):
            outText = outText.replace(p, '')
        else:
            outText = outText.replace(p, ' ')
    outText = period_strip.sub("", outText, re.UNICODE)
    return outText


def process_digit_article(inText):
    outText = []
    tempText = inText.lower().split()
    for word in tempText:
        word = manual_map.setdefault(word, word)
        if word not in articles:
            outText.append(word)
        else:
            pass
    for wordId, word in enumerate(outText):
        if word in contractions:
            outText[wordId] = contractions[word]
    outText = ' '.join(outText)
    return outText


def prep_ans(answer):
    answer = process_digit_article(process_punctuation(answer))
    answer = answer.replace(',', '')
    return answer


def proc_ans(ans):

    ans_prob_dict = {}

    for ans_ in ans:
        ans_proc = prep_ans(ans_['answer'])
        if ans_proc not in ans_prob_dict:
            ans_prob_dict[ans_proc] = 1
        else:
            ans_prob_dict[ans_proc] += 1

    confident_answer = max(ans_prob_dict, key=ans_prob_dict.get)
    return confident_answer

### VQA-X

In [ ]:
"""
From ruotian luo
Create a reference json file used for evaluation with `coco-caption` repo.
Used when multiple evaluation captions exist for a single image
"""
# Explanations only

split = 'test'
in_path = 'nle_data/VQA-X/vqaX_' + split + '.json'
out_path = 'nle_data/VQA-X/vqaX_' + split + '_annot_exp.json'
data = json.load(open(in_path, 'r'))

out = {'info': {'description': 'This is stable 1.0 version of the 2014 MS COCO dataset.', 'url': 'http://mscoco.org', 'version': '1.0', 'year': 2014, 'contributor': 'Microsoft COCO group', 'date_created': '2015-01-27 09:11:52.357475'}, 'licenses': [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}, {'url': 'http://creativecommons.org/licenses/by-nc-nd/2.0/', 'id': 3, 'name': 'Attribution-NonCommercial-NoDerivs License'}, {'url': 'http://creativecommons.org/licenses/by/2.0/', 'id': 4, 'name': 'Attribution License'}, {'url': 'http://creativecommons.org/licenses/by-sa/2.0/', 'id': 5, 'name': 'Attribution-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nd/2.0/', 'id': 6, 'name': 'Attribution-NoDerivs License'}, {'url': 'http://flickr.com/commons/usage/', 'id': 7, 'name': 'No known copyright restrictions'}, {'url': 'http://www.usa.gov/copyright.shtml', 'id': 8, 'name': 'United States Government Work'}], 'type': 'captions'}
out.update({'images': [], 'annotations': []})

cnt = 0

for qid, qid_data in data.items():
    
    out['images'].append({'id': int(qid)})
    
    for s in qid_data['explanation']:
        
        if len(s) == 0:
            print("Warning: {} has no annotations".format(qid))
            continue
            
        out['annotations'].append({'image_id': out['images'][-1]['id'], 'caption': s, 'id': cnt})
        cnt += 1

json.dump(out, open(out_path, 'w'))
print('wrote to ', out_path)

# Explanations + Answers

split = 'test'
in_path = 'nle_data/VQA-X/vqaX_' + split + '.json'
out_path = 'nle_data/VQA-X/vqaX_' + split + '_annot_full.json'

data = json.load(open(in_path, 'r'))

out = {'info': {'description': 'This is stable 1.0 version of the 2014 MS COCO dataset.', 'url': 'http://mscoco.org', 'version': '1.0', 'year': 2014, 'contributor': 'Microsoft COCO group', 'date_created': '2015-01-27 09:11:52.357475'}, 'licenses': [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}, {'url': 'http://creativecommons.org/licenses/by-nc-nd/2.0/', 'id': 3, 'name': 'Attribution-NonCommercial-NoDerivs License'}, {'url': 'http://creativecommons.org/licenses/by/2.0/', 'id': 4, 'name': 'Attribution License'}, {'url': 'http://creativecommons.org/licenses/by-sa/2.0/', 'id': 5, 'name': 'Attribution-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nd/2.0/', 'id': 6, 'name': 'Attribution-NoDerivs License'}, {'url': 'http://flickr.com/commons/usage/', 'id': 7, 'name': 'No known copyright restrictions'}, {'url': 'http://www.usa.gov/copyright.shtml', 'id': 8, 'name': 'United States Government Work'}], 'type': 'captions'}
out.update({'images': [], 'annotations': []})

cnt = 0

for qid, qid_data in data.items():
    
    out['images'].append({'id': int(qid)})
    for s in qid_data['explanation']:
        
        if len(s) == 0:
            print("Warning: {} has no annotations".format(qid))
            continue
            
        s = proc_ans(qid_data['answers']) + " because " + s
        out['annotations'].append({'image_id': out['images'][-1]['id'], 'caption': s, 'id': cnt})
        cnt += 1

json.dump(out, open(out_path, 'w'))
print('wrote to ', out_path)

### ACT-X

In [ ]:
# Explanations

in_path = 'nle_data/ACT-X/actX_test.json'
out_path = 'nle_data/ACT-X/actX_test_annot_exp.json'
data = json.load(open(in_path, 'r'))

out = {'info': {'description': 'This is stable 1.0 version of the 2014 MS COCO dataset.', 'url': 'http://mscoco.org', 'version': '1.0', 'year': 2014, 'contributor': 'Microsoft COCO group', 'date_created': '2015-01-27 09:11:52.357475'}, 'licenses': [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}, {'url': 'http://creativecommons.org/licenses/by-nc-nd/2.0/', 'id': 3, 'name': 'Attribution-NonCommercial-NoDerivs License'}, {'url': 'http://creativecommons.org/licenses/by/2.0/', 'id': 4, 'name': 'Attribution License'}, {'url': 'http://creativecommons.org/licenses/by-sa/2.0/', 'id': 5, 'name': 'Attribution-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nd/2.0/', 'id': 6, 'name': 'Attribution-NoDerivs License'}, {'url': 'http://flickr.com/commons/usage/', 'id': 7, 'name': 'No known copyright restrictions'}, {'url': 'http://www.usa.gov/copyright.shtml', 'id': 8, 'name': 'United States Government Work'}], 'type': 'captions'}
out.update({'images': [], 'annotations': []})

cnt = 0

for imgid, img_data in data.items():
    
    out['images'].append({'id': int(imgid)})
    
    for s in img_data['explanation']:
        
        if len(s) == 0:
            print("Warning: {} has no annotations".format(imgid))
            continue
            
        out['annotations'].append({'image_id': out['images'][-1]['id'], 'caption': s, 'id': cnt})
        cnt += 1

json.dump(out, open(out_path, 'w'))
print('wrote to ', out_path)

# Explanations + Answers

in_path = 'nle_data/ACT-X/actX_test.json'
out_path = 'nle_data/ACT-X/actX_test_annot_full.json'
data = json.load(open(in_path, 'r'))

out = {'info': {'description': 'This is stable 1.0 version of the 2014 MS COCO dataset.', 'url': 'http://mscoco.org', 'version': '1.0', 'year': 2014, 'contributor': 'Microsoft COCO group', 'date_created': '2015-01-27 09:11:52.357475'}, 'licenses': [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/', 'id': 1, 'name': 'Attribution-NonCommercial-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nc/2.0/', 'id': 2, 'name': 'Attribution-NonCommercial License'}, {'url': 'http://creativecommons.org/licenses/by-nc-nd/2.0/', 'id': 3, 'name': 'Attribution-NonCommercial-NoDerivs License'}, {'url': 'http://creativecommons.org/licenses/by/2.0/', 'id': 4, 'name': 'Attribution License'}, {'url': 'http://creativecommons.org/licenses/by-sa/2.0/', 'id': 5, 'name': 'Attribution-ShareAlike License'}, {'url': 'http://creativecommons.org/licenses/by-nd/2.0/', 'id': 6, 'name': 'Attribution-NoDerivs License'}, {'url': 'http://flickr.com/commons/usage/', 'id': 7, 'name': 'No known copyright restrictions'}, {'url': 'http://www.usa.gov/copyright.shtml', 'id': 8, 'name': 'United States Government Work'}], 'type': 'captions'}
out.update({'images': [], 'annotations': []})

cnt = 0

for imgid, img_data in data.items():
    
    out['images'].append({'id': int(imgid)})
    
    for s in img_data['explanation']:
        
        if len(s) == 0:
            print("Warning: {} has no annotations".format(imgid))
            continue
            
        s = prep_ans(img_data['answers']) + " because " + s
        out['annotations'].append({'image_id': out['images'][-1]['id'], 'caption': s, 'id': cnt})
        cnt += 1

json.dump(out, open(out_path, 'w'))
print('wrote to ', out_path)

### eSNLI-VE

In [ ]:
# Explanations 

in_path = 'nle_data/eSNLI-VE/esnlive_test.json'
out_path = 'nle_data/eSNLI-VE/esnlive_test_annot_exp.json'
data = json.load(open(in_path, 'r'))

test = {
    'images': [],
    'annotations': [],
    'info': {'description': 'eSNLI-VE (test split)'},
    'type': 'captions',
    'licenses': 'http://creativecommons.org/licenses/by/4.0/',
}

unique_ids = []

for imgid, (key,value) in enumerate(data.items()):
    
    # Extract info
    url           = 'https://github.com/maximek3/e-ViL'             
    filename      = value['image_name']
    id            = int(key)                    
    explanation   = value['explanation']

    # Skip duplicate paragraph captions
    if id in unique_ids: 
        print("Found duplicate!")
        break   # this format assumes only 1 caption per image, otherwise cider calculation goes wrong
    else:
        unique_ids.append(id)

    # Extract image info
    image = {
        'url': url,
        'file_name': filename,
        'id': id,
    }

    # Extract caption info
    annotation = {
        'image_id': id,
        'id': imgid,
        'caption': explanation
    }

    # Store info
    test['images'].append(image)
    test['annotations'].append(annotation)
    
json.dump(test, open(out_path, 'w'))
print('wrote to ', out_path)

# Explanations + Answers

in_path = 'nle_data/eSNLI-VE/esnlive_test.json'
out_path = 'nle_data/eSNLI-VE/esnlive_test_annot_full.json'
data = json.load(open(in_path, 'r'))

test = {
    'images': [],
    'annotations': [],
    'info': {'description': 'eSNLI-VE (test split)'},
    'type': 'captions',
    'licenses': 'http://creativecommons.org/licenses/by/4.0/',
}

unique_ids = []

for imgid, (key,value) in enumerate(data.items()):
    
    # Extract info
    url           = 'https://github.com/maximek3/e-ViL'             
    filename      = value['image_name']
    id            = int(key)                    
    explanation   = value['answers'] + " because " + value['explanation']

    # Skip duplicate paragraph captions
    if id in unique_ids: 
        print("Found duplicate!")
        break   # this format assumes only 1 caption per image, otherwise cider calculation goes wrong
    else:
        unique_ids.append(id)

    # Extract image info
    image = {
        'url': url,
        'file_name': filename,
        'id': id,
    }

    # Extract caption info
    annotation = {
        'image_id': id,
        'id': imgid,
        'caption': explanation
    }

    # Store info
    test['images'].append(image)
    test['annotations'].append(annotation)
    
json.dump(test, open(out_path, 'w'))
print('wrote to ', out_path)

### VCR

In [ ]:
# Explanations 

in_path = 'vcr_test.json'
out_path = 'vcr_test_annot_exp.json'
data = json.load(open(in_path, 'r'))

test = {
    'images': [],
    'annotations': [],
    'info': {'description': 'VCR (test split)'},
    'type': 'captions',
    'licenses': 'http://creativecommons.org/licenses/by/4.0/',
}

unique_ids = []

for imgid, (key,value) in enumerate(data.items()):
    
    # Extract info
    url           = 'https://github.com/maximek3/e-ViL'             
    filename      = value['img_name']
    id            = int(key)                    
    explanation   = value['explanation']

    # Skip duplicate paragraph captions
    if id in unique_ids: 
        print("Found duplicate!")
        break   # this format assumes only 1 caption per image, otherwise cider calculation goes wrong
    else:
        unique_ids.append(id)

    # Extract image info
    image = {
        'url': url,
        'file_name': filename,
        'id': id,
    }

    # Extract caption info
    annotation = {
        'image_id': id,
        'id': imgid,
        'caption': explanation
    }

    # Store info
    test['images'].append(image)
    test['annotations'].append(annotation)
    
json.dump(test, open(out_path, 'w'))
print('wrote to ', out_path)

# Explanations + Answers

in_path = 'vcr_test.json'
out_path = 'vcr_test_annot_full.json'
data = json.load(open(in_path, 'r'))

test = {
    'images': [],
    'annotations': [],
    'info': {'description': 'eSNLI-VE (test split)'},
    'type': 'captions',
    'licenses': 'http://creativecommons.org/licenses/by/4.0/',
}

unique_ids = []

for imgid, (key,value) in enumerate(data.items()):
    
    # Extract info
    url           = 'https://github.com/maximek3/e-ViL'             
    filename      =  value['img_name']
    id            =  int(key)                    
    explanation   =  value['answers'] + " because " + value['explanation']

    # Skip duplicate paragraph captions
    if id in unique_ids: 
        print("Found duplicate!")
        break   # this format assumes only 1 caption per image, otherwise cider calculation goes wrong
    else:
        unique_ids.append(id)

    # Extract image info
    image = {
        'url': url,
        'file_name': filename,
        'id': id,
    }

    # Extract caption info
    annotation = {
        'image_id': id,
        'id': imgid,
        'caption': explanation
    }

    # Store info
    test['images'].append(image)
    test['annotations'].append(annotation)
    
json.dump(test, open(out_path, 'w'))
print('wrote to ', out_path)